In [59]:
from PIL import Image
import numpy as np
import json
import matplotlib.pyplot as plt
from collections import Counter
from typing import Dict, Any, Tuple, List
import cv2

SCAN_DIR = "../res/scans"
OUT_DIR = "../res/out"

In [60]:
views_list = []
with open(f"{SCAN_DIR}/views.json") as f:
    for line in f:
        views_list.append(
            json.loads(line.strip())
        )

views_list = views_list[:1000]

In [ ]:
# NOTE: we use PIL to loead images to remain consistent with the rest of the code
FPS = 30
# max depth in mm
MAX_DEPTH = 5000.

col_test_frame = np.array(Image.open(f"{SCAN_DIR}/{views_list[0]["file_name"]}"))
ch, cw, _ = col_test_frame.shape
depth_test_frame = np.array(Image.open(f"{SCAN_DIR}/{views_list[0]["depth_file_name"]}"))
dh, dw = depth_test_frame.shape

col_video = cv2.VideoWriter(
    f"{OUT_DIR}/scan_video.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (cw, ch),
)
depth_video = cv2.VideoWriter(
    f"{OUT_DIR}/depth_scan_video.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    FPS,
    (dw, dh),
)

for view in views_list:
    # load and write color frame
    col_frame = np.array(Image.open(f"{SCAN_DIR}/{view["file_name"]}"))
    col_video.write(col_frame) # TODO: Fix -> PIL versus cv2 rgb encoding is slightly different bcuz vid is very blue lol
    
    # load grayscale image as RGB and scale depth frame
    depth_frame = np.atleast_3d(Image.open((f"{SCAN_DIR}/{view["depth_file_name"]}"))).repeat(3, axis=-1)
    depth_frame = np.clip(depth_frame / MAX_DEPTH * 255., 0., 255.)
    # set 0 pixels (i.e., invalid pixels) to salmon pink
    depth_1D = depth_frame.reshape(-1, 3)
    zero_inds = np.where(depth_1D == np.array([0, 0, 0]))[0]
    depth_1D[zero_inds] = np.array([164, 145, 255])
    depth_frame = depth_1D.reshape(dh, dw, 3)

    # write depth frame
    depth_video.write(depth_frame.astype(np.uint8))

cv2.destroyAllWindows()
col_video.release()
depth_video.release()